In [10]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

from calitp_data_analysis.tables import tbls

import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils

In [17]:
stops = (tbls.mart_gtfs.fct_daily_scheduled_stops()
    >> filter(_.service_date == '2023-09-13')
    >> select(_.service_date, _.feed_key, _.stop_id,
              _.pt_geom)
    >> collect()
)

/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


In [18]:
stops = gpd.GeoDataFrame(stops,
                         geometry=gpd.geoseries.from_wkt(stops['pt_geom'].to_numpy()),
                         crs='EPSG:4326') >> select(-_.pt_geom)

In [19]:
stops >> head(3)

,service_date,feed_key,stop_id,geometry
0,2023-09-13,b45b85ee42cd6a4311beab4aaf7e09d8,29340,POINT (-120.46339 36.86854)
1,2023-09-13,b45b85ee42cd6a4311beab4aaf7e09d8,29343,POINT (-120.45270 36.85343)
2,2023-09-13,b45b85ee42cd6a4311beab4aaf7e09d8,29344,POINT (-120.45195 36.85275)


In [12]:
stops.shape

(119150, 4)

In [13]:
# (stops >> head(1000) >> select(-_.service_date)).explore()

In [1]:
! pip install pygris

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 873.6 kB/s eta 0:00:00 0:00:01


In [2]:
import pygris

In [7]:
uza = pygris.urban_areas()

Using the default year of 2021


In [31]:
ca_uza = uza >> filter(_.NAME10.str.contains('CA'))

In [13]:
ca_tracts = pygris.tracts(state='CA', year=2020)

ca_tracts = ca_tracts >> select(_.GEOID, _.geometry)

ca_tracts = ca_tracts.to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [20]:
stops = stops.to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [21]:
stops

,service_date,feed_key,stop_id,geometry
0,2023-09-13,b45b85ee42cd6a4311beab4aaf7e09d8,29340,POINT (-41252.686 -127493.402)
1,2023-09-13,b45b85ee42cd6a4311beab4aaf7e09d8,29343,POINT (-40309.757 -129177.891)
2,2023-09-13,b45b85ee42cd6a4311beab4aaf7e09d8,29344,POINT (-40242.891 -129253.681)
3,2023-09-13,b45b85ee42cd6a4311beab4aaf7e09d8,29347,POINT (-34017.110 -140003.552)
4,2023-09-13,b45b85ee42cd6a4311beab4aaf7e09d8,29348,POINT (-34004.854 -139993.376)
...,...,...,...,...
119145,2023-09-13,7519b1626b959b0fc517692b19431065,2611973,POINT (-83917.665 72237.829)
119146,2023-09-13,7519b1626b959b0fc517692b19431065,782799,POINT (-92741.321 70695.418)
119147,2023-09-13,7519b1626b959b0fc517692b19431065,782761,POINT (-72120.851 76963.261)
119148,2023-09-13,7519b1626b959b0fc517692b19431065,782795,POINT (-86800.440 71821.263)


In [23]:
stops_in_tract = gpd.sjoin(ca_tracts, stops)

In [24]:
tracts_and_feeds = stops_in_tract >> distinct(_.GEOID, _.geometry, _.feed_key)

In [25]:
tracts_and_feeds

,GEOID,geometry,feed_key
0,06029004402,"POLYGON ((59906.842 -270407.558, 59906.948 -27...",88fde0226aecc4b91a3900f52214f03b
1,06029004402,"POLYGON ((59906.842 -270407.558, 59906.948 -27...",c13333e968e573c9decd68bc7fc2555d
2,06047000802,"POLYGON ((-52727.332 -74888.681, -52723.908 -7...",7a4bf0aaccd7aa130db46358ebdb5c26
3,06085501402,"POLYGON ((-165759.786 -73091.934, -165741.347 ...",371141abbbf8d5e3a42694a3a3959747
4,06085501402,"POLYGON ((-165759.786 -73091.934, -165741.347 ...",eb9f5e395e946e3e6b8ed4b43c820e72
...,...,...,...
14807,06013338101,"POLYGON ((-180031.401 -5698.046, -179998.229 -...",fbf82b1f92575cb511fd97db46a0562a
14808,06013338101,"POLYGON ((-180031.401 -5698.046, -179998.229 -...",a80da65523070e82301e8990873fd9c4
14809,06013338102,"POLYGON ((-180162.918 -6902.976, -180150.812 -...",fbf82b1f92575cb511fd97db46a0562a
14810,06013338102,"POLYGON ((-180162.918 -6902.976, -180150.812 -...",a80da65523070e82301e8990873fd9c4


In [28]:
# (tracts_and_feeds >> distinct(_.geometry)).explore()

In [26]:
tracts_and_feeds.to_parquet('intermediate/feeds_tract_geo.parquet')

In [34]:
ca_uza = ca_uza >> select(_.UACE10, _.NAME10, _.geometry)
ca_uza = ca_uza.to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [35]:
ca_uza.to_parquet('intermediate/ca_uza.parquet')